##### Copyright 2018 The TensorFlow Authors.

Licensed under the Apache License, Version 2.0 (the "License");

In [0]:
#@title Licensed under the Apache License, Version 2.0 (the "License"); { display-mode: "form" }
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Cox Process with TFP

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://drive.google.com/file/d/1LW22s427IS2XaMFpb6Rf_W4rVjaKfXBi/view?usp=sharing"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href=""><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
</table>
<br>
<br>
<br>

Original content [this Repository](https://github.com/blei-lab/edward/blob/master/examples/cox_process.py), created by [the Blei Lab](http://www.cs.columbia.edu/~blei/)

Ported to Tensorflow Probability by Matthew McAteer ([`@MatthewMcAteer0`](https://twitter.com/MatthewMcAteer0)), with help from the TFP team at  Google ([`tfprobability@tensorflow.org`](mailto:tfprobability@tensorflow.org)).

---

>[Dependencies & Prerequisites](#scrollTo=2ZtWUjXYRXQi)

>[Introduction](#scrollTo=2ZtWUjXYRXQi)

>>[Data](#scrollTo=2ZtWUjXYRXQi)

>>[Model](#scrollTo=2ZtWUjXYRXQi)

>>[Inference](#scrollTo=2ZtWUjXYRXQi)

>>[Criticism](#scrollTo=2ZtWUjXYRXQi)

>[References](#scrollTo=2ZtWUjXYRXQi)


## Dependencies & Prerequisites

In [0]:
!pip3 install -q tfp-nightly
!pip3 install -q observations

In [0]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

# import edward as ed
import numpy as np
import tensorflow as tf
import tensorflow_probability as tfp

tfd = tfp.distributions

# from edward.models import MultivariateNormalTriL, Normal, Poisson
# from edward.util import rbf
from scipy.stats import multivariate_normal, poisson

In [0]:
def session_options(enable_gpu_ram_resizing=True, enable_xla=True):
    """
    Allowing the notebook to make use of GPUs if they're available.
    
    XLA (Accelerated Linear Algebra) is a domain-specific compiler for linear 
    algebra that optimizes TensorFlow computations.
    """
    config = tf.ConfigProto()
    config.log_device_placement = True
    if enable_gpu_ram_resizing:
        # `allow_growth=True` makes it possible to connect multiple colabs to your
        # GPU. Otherwise the colab malloc's all GPU ram.
        config.gpu_options.allow_growth = True
    if enable_xla:
        # Enable on XLA. https://www.tensorflow.org/performance/xla/.
        config.graph_options.optimizer_options.global_jit_level = (
            tf.OptimizerOptions.ON_1)
    return config


def reset_sess(config=None):
    """
    Convenience function to create the TF graph & session or reset them.
    """
    if config is None:
        config = session_options()
    global sess
    tf.reset_default_graph()
    try:
        sess.close()
    except:
        pass
    sess = tf.InteractiveSession(config=config)

    
def evaluate(tensors):
    """
    A "Universal" evaluate function for both running either Graph mode (default)
    or Eager mode (https://www.tensorflow.org/guide/eager) in Tensorflow.
    """
    if context.executing_eagerly():
        return (t.numpy() for t in tensprs)
    with tf.get_default_session() as sess:
        return sess.run(tensors)

reset_sess()


def strip_consts(graph_def, max_const_size=32):
  """
  Strip large constant values from graph_def.
  """
  strip_def = tf.GraphDef()
  for n0 in graph_def.node:
    n = strip_def.node.add()
    n.MergeFrom(n0)
    if n.op == 'Const':
      tensor = n.attr['value'].tensor
      size = len(tensor.tensor_content)
      if size > max_const_size:
        tensor.tensor_content = bytes("<stripped %d bytes>"%size, 'utf-8')
  return strip_def


def draw_graph(model, *args, **kwargs):
  """
  Visualize TensorFlow graph.
  """
  graph = tf.Graph()
  with graph.as_default():
    model(*args, **kwargs)
  graph_def = graph.as_graph_def()
  strip_def = strip_consts(graph_def, max_const_size=32)
  code = """
      <script>
        function load() {{
          document.getElementById("{id}").pbtxt = {data};
        }}
      </script>
      <link rel="import" href="https://tensorboard.appspot.com/tf-graph-basic.build.html" onload=load()>
      <div style="height:600px">
        <tf-graph-basic id="{id}"></tf-graph-basic>
      </div>
  """.format(data=repr(str(strip_def)), id='graph'+str(np.random.rand()))

  iframe = """
      <iframe seamless style="width:1200px;height:620px;border:0" srcdoc="{}"></iframe>
  """.format(code.replace('"', '&quot;'))
  IPython.display.display(IPython.display.HTML(iframe))

A Cox process model for spatial analysis (Cox, 1955; Miller et al., 2014). The data set is a $N x V$ matrix. There are N NBA players, X = {(x_1, ..., x_N)}, where each x_n has a set of V counts. x_{n, v} is
the number of attempted basketball shots for the $n$th NBA player at
location $v$.
We model a latent intensity function for each data point. Let $K$ be the
$N x V x V$ covariance matrix applied to the data set $X$ with fixed
kernel hyperparameters, where a slice $K_n$ is the $V x V$ covariance
matrix over counts for a data point $x_n$.

For $n = 1, ..., N $,

$$ p(f_n) = N(f_n | 0, K_n) $$,

$$ p(x_n | f_n) = \prod_{v=1}^V p(x_{n,v} | f_{n,v}) $$,

$$ \text{where } p(x_{n,v} | f_{n, v}) = \text{Poisson}(x_{n,v} | exp(f_{n,v})) $$.

In [0]:
# tf.flags.DEFINE_integer("N", default=308, help="Number of NBA players.")
# tf.flags.DEFINE_integer("V", default=2, help="Number of shot locations.")
# FLAGS = tf.flags.FLAGS

N = 308   # Number of NBA players
V = 2     # Number of shot locations


In [0]:
def build_toy_dataset(N, V):
    """
    A simulator mimicking the data set from 2015-2016 NBA season with
    308 NBA players and ~150,000 shots.
    """
    L = np.tril(np.random.normal(2.5, 0.1, size=[V, V]))
    K = np.matmul(L, L.T)
    x = np.zeros([N, V])
    for n in range(N):
        f_n = multivariate_normal.rvs(cov=K, size=1)
        for v in range(V):
            x[n, v] = poisson.rvs(mu=np.exp(f_n[v]), size=1)

    return x


### Data

In [0]:
# ed.set_seed(42)

x_data = build_toy_dataset(FLAGS.N, FLAGS.V)

### Model

In [0]:
x_ph = tf.placeholder(tf.float32, [N, V])

# Form (N, V, V) covariance, one matrix per data point.
K = tf.stack([rbf(tf.reshape(xn, [V, 1])) + tf.diag([1e-6, 1e-6])
                for xn in tf.unstack(x_ph)])
f = tfd.MultivariateNormalTriL(loc=tf.zeros([N, V]),
                             scale_tril=tf.cholesky(K))
x = tfd.Poisson(rate=tf.exp(f))

### Inference

In [0]:
qf = tfd.Normal(
    loc=tf.get_variable("qf/loc", [N, V]),
    scale=tf.nn.softplus(tf.get_variable("qf/scale", [N, V])))

inference = ed.KLqp({f: qf}, data={x: x_data, x_ph: x_data})
inference.run(n_iter=5000)

In [0]:
# Visualizing the graph we've constructed
# draw_graph(linear_mixed_effects_model, features_train)

## References

1. A Cox process model for spatial analysis (Cox, 1955; Miller et al., 2014).

In [0]:
from IPython.core.display import HTML
def css_styling():
    styles = open("../styles/custom.css", "r").read()
    return HTML(styles)
css_styling()

#  "#F15854",  // red
#  "#5DA5DA",  // blue
#  "#FAA43A",  // orange
#  "#60BD68",  // green
#  "#F17CB0",  // pink
#  "#B2912F",  // brown
#  "#B276B2",  // purple
#  "#DECF3F",  // yellow
#  "#4D4D4D",  // gray